In [104]:
import rasterio
import numpy as np
import cv2
from os import listdir
import pandas as pd

In [105]:
def get_rgb_geotiff(file_path, r_band=1, g_band=2, b_band=3, ik_band=4, mask_band=5, chanel=1):
    try:
        with rasterio.open(file_path) as src:
            red = src.read(r_band)  # B02 - Blue
            green = src.read(g_band)  # B03 - Green
            blue = src.read(b_band)  # B04 - Red
            ik = src.read(ik_band)
            mask = src.read(mask_band)

            if chanel == 1:
                photo = np.clip(np.stack([red, green, blue], axis=-1).astype(float) * 3, 0, 255).astype(np.uint8)  # Отрисовка всего изображения
            elif chanel == 2:
                photo = np.stack([ik], axis=-1) # Отрисовка ИК-слоя изображения
            elif chanel == 3:
                photo = np.stack([mask], axis=-1) * 255  # Отрисовка маски изображения
            photo = photo.astype(np.uint8)
            return photo
    except Exception as e:
        print(f'Ошибка: {e}')


def get_rgb_geotiff_data(file_path):
    try:
        with rasterio.open(file_path) as src:            
            top_left = (src.transform * (0, 0))
            bottom_right = (src.transform * (src.width, src.height))

            middle_x = (top_left[0] + bottom_right[0]) / 2
            middle_y = (top_left[1] + bottom_right[1]) / 2

            return middle_x, middle_y
    except Exception as e:
        print(f'Ошибка: {e}')

In [106]:
df = []
df_weather = [] 

for i, folder in enumerate(listdir('temp')):
    file = listdir(f"temp/{folder}")[0]
    path = f'temp/{folder}/{file}'

    temp = pd.read_csv(f"temp/{folder}/{listdir(f'temp/{folder}')[1]}")
    temp['file'] = f'image_{i + 1}.jpg'
    df_weather.append(temp.reset_index(drop=True))

    data = '.'.join(file.split('.')[:-1])
    middle_x, middle_y = get_rgb_geotiff_data(path)
    df.append([f'image_{i + 1}.jpg', data, middle_x, middle_y])

    cv2.imwrite(f'data/image/image_{i + 1}.jpg', get_rgb_geotiff(path, chanel=1))
    cv2.imwrite(f'data/ik/image_{i + 1}.jpg', get_rgb_geotiff(path, chanel=2))
    cv2.imwrite(f'data/mask/image_{i + 1}.jpg', get_rgb_geotiff(path, chanel=3))


In [107]:
pd.DataFrame(df, columns=['file', 'date', 'middle_x', 'middle_y']).to_csv("data/metadata.csv", index=None)

In [108]:
temp = pd.concat(df_weather).dropna(how='all', axis=1)
temp['Атмосферное Давление'] = temp['Атмосферное Давление'].fillna(temp['Давление на уровне моря'])
temp.drop(['Давление на уровне моря'], axis=1, inplace=True)
temp['time'] = pd.to_datetime(temp['time']).astype('int64') // 10 ** 11 # ! ПОМЕНЯТЬ ДЛЯ ТЕСТОВЫХ
temp.fillna(0).to_csv("data/weather.csv", index=None)